# Coin Accounter System 

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import cv2 as cv
import glob
import time

In [2]:
# Função para facilitar a visualização das imagens
def showit(img, openCV=0, text='t'):
    
    if(openCV):
        cv.imshow(text, img[::2,::2])
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    else:  
        # Para imagens preto e branco
        if(len(img.shape) < 3):
            plt.gray()
            plt.imshow(img)
            plt.show()
        else:
            plt.imshow(img[:,:,::-1])
            plt.show()

### preprocessing images

In [3]:
def improveTexture(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    return img

def resize(img, cols=800):    
    d = cols / img.shape[1]
    dim = (cols, int(img.shape[0] * d))
    img = cv.resize(img, dim, interpolation=cv.INTER_AREA)
    return img

### Getting Coin ROI

In [4]:
# Função que retorna a mascara das moedas na imagem
def getMask(img):
    
    a, b = -1/4, 4
    kernel = np.array([[a,a,a],[a,b,a],[a,a,a]])
    img = cv.filter2D(img,-1,kernel)
    img = cv.bitwise_not(img, img)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    limiar, img = cv.threshold(img, 30, 255, cv.THRESH_BINARY) # limiar de otsu inclui as sombras
    
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (13,13))
    mask = cv.morphologyEx(img, cv.MORPH_CLOSE, stt, iterations=5)
    
    return mask

# Função que encontrar a moeda na imagem e retorna o menor quadrado envolvente com a moeda
def findSingleCoin(imgCol):
    
    # Reduz tamanho da imagem
    imgCol = resize(imgCol, 800)
    
#     showit(imgCol, 1)
    
    # Encontro a Região de Interesse
    imgBin = getMask(imgCol)

    # Selecionando região de interesse na imagem colorida com a máscara encontrada
    imgCol = cv.bitwise_and(imgCol, imgCol, mask=imgBin)
    
#     showit(imgCol, 1)
    
    # Achar contornos das moedas
    moedas, hierarquia = cv.findContours(imgBin, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        
    # Para cada moeda encontrada
    for moeda in moedas:
        
        # Não contabilizar regiões de falhas
        area = cv.contourArea(moeda)
        if(area < 5000.0):            
            continue
            
        # Encontre o menor retângulo envolvente
        x, y, w, h = cv.boundingRect(moeda)

        # Nova imagem com o menor retângulo envolvente da moeda
        imgCoinRoi = imgCol[y:y+h, x:x+w]

#         showit(imgCoinRoi)
        
        return imgCoinRoi

### Extractions Functions Routines

In [5]:
def lbp(img):
    qtdeLinhas, qtdeColunas = img.shape
    out = img.copy()
    
    for i in range(1,qtdeLinhas-1):
        for j in range(1,qtdeColunas-1):                
            a=img[i-1,j]
            b=img[i-1,j+1]
            c=img[i,j+1]
            d=img[i+1,j+1]
            e=img[i+1,j]
            f=img[i+1,j-1]
            g=img[i,j-1]
            h=img[i-1,j-1]
            centro=img[i,j]
            
            soma=0
            
            if(a<centro):
                soma = soma + 2**7
            if(b<centro):
                soma = soma + 2**6
            if(c<centro):
                soma = soma + 2**5
            if(d<centro):
                soma = soma + 2**4
            if(e<centro):
                soma = soma + 2**3
            if(f<centro):
                soma = soma + 2**2
            if(g<centro):
                soma = soma + 2**1
            if(h<centro):
                soma = soma + 2**0

            out[i,j]=soma
    
#     showit(out, 1, 'lbp')
    
    return out

In [6]:
def saturationHistogram(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)    
    hist = cv.calcHist([img],[0],None,[256],[0,256])
    hist = cv.normalize(hist, None)
    
    return hist.flatten()

In [7]:
def coloredHistogram(img):
    
    histBlue = cv.calcHist([img[:,:,0]],[0],None,[256],[0,256])
    histBlue = cv.normalize(histBlue, None)
    
    histGreen = cv.calcHist([img[:,:,1]],[0],None,[256],[0,256])
    histGreen = cv.normalize(histGreen, None)
    
    histRed = cv.calcHist([img[:,:,2]],[0],None,[256],[0,256])
    histRed = cv.normalize(histRed, None)
    
    for i in range(len(histBlue)):
        histBlue[i] = min(histBlue[i], histGreen[i], histRed[i])
        
    return histBlue.flatten()

In [8]:
def histograma(img):
    hist = cv.calcHist([img],[0],None,[256],[0,256])
    hist = cv.normalize(hist, None)
    return hist.flatten()

In [9]:
def extraction(imag, roiDefined=False):
    
    if(not roiDefined):
        imag = findSingleCoin(imag)

    color = coloredHistogram(imag)
    saturation = saturationHistogram(imag)

    imag = improveTexture(imag)
    imag = lbp(imag)
    texture = histograma(imag)
    
    return np.append(np.append(texture, color), saturation)

def extractionFromFile(file):
    img = cv.imread(file)
    return extraction(img)

## Set Train Dataset

In [10]:
import pickle
from os.path import isfile as exist
start_time = time.time()

class Enum(tuple):__getattribute__ = tuple.index

# ETAPA 1: EXTRAIR CARACTERÍSTICAS DAS IMAGENS
Coins = Enum(("CINCO", "DEZ", "VINTECINCO", "CINQUENTA", "REAL"))

if(exist("coin_features") and exist("coin_features_answer")):
    
    with open('coin_features', 'rb') as f:
        data = pickle.load(f)

    with open('coin_features_answer', 'rb') as f:
        answrData = pickle.load(f)

else:
    
    # Defina onde está a pasta do dataset da moeda
    coin_directory = "datacoins/"

    # Lista com o nome das imagens nesse diretório
    sample_images_5_back = glob.glob(coin_directory + "5/back/*")
    sample_images_5_front = glob.glob(coin_directory + "5/front/*")

    sample_images_10_back = glob.glob(coin_directory  + "10/back/*")
    sample_images_10_front = glob.glob(coin_directory + "10/front/*")

    sample_images_25_back = glob.glob(coin_directory  + "25/back/*")
    sample_images_25_front = glob.glob(coin_directory + "25/front/*")

    sample_images_50_back = glob.glob(coin_directory  + "50/back/*")
    sample_images_50_front = glob.glob(coin_directory + "50/front/*")

    sample_images_100_back = glob.glob(coin_directory  + "100/back/*")
    sample_images_100_front = glob.glob(coin_directory + "100/front/*")

    # Lista com as características das imagens e suas respectivas classificações
    data = []
    answrData = []
    arffData = []
    classes = ['0', '1', '2', '3', '4']

    # Extraindo características das moedas sozinhas
    for i in sample_images_5_front:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.CINCO)

        arffData.append((features, str(Coins.CINCO)))

    for i in sample_images_5_back:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.CINCO)

        arffData.append((features, str(Coins.CINCO)))

    for i in sample_images_10_front:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.DEZ)

        arffData.append((features, str(Coins.DEZ)))

    for i in sample_images_10_back:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.DEZ)

        arffData.append((features, str(Coins.DEZ)))

    for i in sample_images_25_front:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.VINTECINCO)

        arffData.append((features, str(Coins.VINTECINCO)))

    for i in sample_images_25_back:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.VINTECINCO)

        arffData.append((features, str(Coins.VINTECINCO)))

    for i in sample_images_50_front:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.CINQUENTA)

        arffData.append((features, str(Coins.CINQUENTA)))

    for i in sample_images_50_back:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.CINQUENTA)

        arffData.append((features, str(Coins.CINQUENTA)))

    for i in sample_images_100_front:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.REAL)

        arffData.append((features, str(Coins.REAL))) 

    for i in sample_images_100_back:
        features = extractionFromFile(i)
        data.append(features)
        answrData.append(Coins.REAL)

        arffData.append((features, str(Coins.REAL)))

    # Save the results
    with open('coin_features', 'wb') as f:
        pickle.dump(data, f)

    with open('coin_features_answer', 'wb') as f:
        pickle.dump(answrData, f)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00603485107421875 seconds ---


## Classifier

In [11]:
from sklearn.model_selection import train_test_split

# 75% para treino, 25% da medir a accuracia
data_train, data_test, asw_train, asw_test = train_test_split(
    data, answrData, test_size=.25)

In [12]:
from sklearn.tree import DecisionTreeRegressor

trReg = DecisionTreeRegressor().fit(data_train,asw_train)
# t.predict(test)

score = int(trReg.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
trReg = DecisionTreeRegressor().fit(data,answrData)

Classifier mean accuracy:  89 %


In [13]:
from sklearn.neural_network import MLPClassifier
start_time = time.time()

mlp = MLPClassifier(solver="lbfgs").fit(data_train,asw_train)

score = int(mlp.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
mlp = MLPClassifier(solver="lbfgs").fit(data,answrData)

print("--- %s seconds ---" % (time.time() - start_time))

Classifier mean accuracy:  95 %
--- 15.949693202972412 seconds ---


In [14]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50).fit(data_train,asw_train)
# clf.predict(test)

score = int(clf.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
clf = RandomForestClassifier(n_estimators=50, bootstrap = True, max_features = 'sqrt').fit(data_train,asw_train)

Classifier mean accuracy:  96 %


In [15]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier().fit(data_train,asw_train)
# tree.predict()

score = int(tree.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
tree = DecisionTreeClassifier().fit(data_train,asw_train).fit(data,answrData)

Classifier mean accuracy:  94 %


In [27]:
# classifier = trReg
# classifier = tree
classifier = clf
# classifier = mlp

## Data Input

In [49]:
def predictCoin(roi):
    hist = extraction(roi, True)
    s = classifier.predict([hist])
    
    # Lista com as probabilidade de cada moeda
    a = classifier.predict_proba([hist]).tolist()
        
    return Coins[a[0].index(max(a[0]))], max(a[0])

def getMask(img, RoiDefined=False):
    
    a, b = -1/4, 4
    kernel = np.array([[a,a,a],[a,b,a],[a,a,a]])
    img = cv.filter2D(img,-1,kernel)
    img = cv.bitwise_not(img, img)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    limiar, img = cv.threshold(img, 30, 255, cv.THRESH_BINARY) # limiar de otsu inclui as sombras
    
    if(RoiDefined):
        size = 7
    else:
        size = 31
    
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (size,size))
    mask = cv.morphologyEx(img, cv.MORPH_CLOSE, stt, iterations=1)
    
    return mask

def findMultiplesCoin(imgCol):
    
    imgBin = getMask(imgCol)
    imgCol = cv.bitwise_and(imgCol, imgCol, mask=imgBin)
    
    img = cv.cvtColor(imgCol, cv.COLOR_BGR2GRAY)
    img = cv.blur(img, (9,9))
    
#     showit(img, 1)
    
    circles = cv.HoughCircles(img,method=cv.HOUGH_GRADIENT,dp=1,minDist=100,
                               param1=50,param2=50,minRadius=40,maxRadius=80)
    
    return circles

# Imagem que será analisada
img = cv.imread("datacoins/juntas/10front-10front-25front-5front-5back-10back-10front.jpg")
img = resize(img, 800)

output = img.copy()

circles = findMultiplesCoin(img)

count = 0
if circles is not None:
    # coordinates and radii
    circles = np.round(circles[0,:]).astype(int)
    
    for (x, y, d) in circles:
        count += 1
        d += 5
                
        roi = img[y-d:y+d, x-d:x+d]
        
        imgBinRoi = getMask(roi, True)
        roi = cv.bitwise_and(roi, roi, mask=imgBinRoi)
    
        showit(roi, 1)
        
        coin, chance = predictCoin(roi)
        
        chance = str(int(chance*100)) + " %"
        
        cv.circle(output, (x,y), d, (0, 255, 0), 2)
        cv.putText(output, coin, (x - 40, y), cv.FONT_HERSHEY_PLAIN,
                   1.5, (0, 255, 0), thickness=2, lineType=cv.LINE_AA)
        
        cv.putText(output, chance, (x - 20, y+30), cv.FONT_HERSHEY_PLAIN,
                   1.5, (0, 255, 0), thickness=2, lineType=cv.LINE_AA)

showit(output, 1)

In [18]:
#     height,width,depth = im.shape
# circle_img = np.zeros((height,width), np.uint8)
# cv2.circle(circle_img,(width//2,height//2),height//2,255,thickness=-1)
# masked_data = cv2.bitwise_and(im, im, mask=circle_img)

## Arquivo ARFF para testar no Keras

In [19]:
from os.path import isfile as exist

def gravar_arquivo_arff(base_teste, classes):
    tam = len(base_teste[0])
    file = open('data.arff','w') 
 
    file.write('@relation teste\n') 
    for i in range(tam):
        file.write('@attribute '+ str(i) +' NUMERIC\n') 
    
    file.write('@attribute classes {')
    
    a = set(classes)
    
    for i in a:
        file.write(str(i)+',')
    
    file.write('}')    
    for i in range(tam):
         len(set(classes))
    
    file.write('\n@data\n') 

    for item in base_teste:
        for i in range(len(item)):
            file.write("%s," % str(item[0][i])) 
        file.write("%s\n" % item[1])    
 
    file.close()
    
if(not exist("data.arff")):
    gravar_arquivo_arff(arffData,classes)